# Benchmarking Demo

This notebook demonstrates the use of the Confidential Benchmarking tool. It requires the [avato Benchmarking API](https://github.com/decentriq/avato-python-client-benchmarking) and its dependencies to be installed.  

Note that in a realistic, non-demo use of the Confidential Benchmarking tool, one admin user and multiple participant users would upload data from different computers. In this workbook, for simplicity, the workflows of the admin user and the two participant users are all shown together.   

### Import dependencies

In [ ]:
import os
import json
from avato import Client
from avato import Secret
from avato_training import Training_Instance, Configuration

admin_username = "***REMOVED***"
admin_***REMOVED*** = "***REMOVED***"

participant1_username = "***REMOVED***"
participant1_***REMOVED*** = "***REMOVED***"

participant2_username = "***REMOVED***"
participant2_***REMOVED*** = "***REMOVED***"

# This is the hash of the code
expected_measurement = "4ff505f350698c78e8b3b49b8e479146ce3896a06cd9e5109dfec8f393f14025"

backend_host = "localhost" # "avato-staging.westeurope.cloudapp.azure.com"
backend_port = 3000 # 15005

### ADMIN USER - Set Up Instance
#### Create new instance

In [ ]:
# Create client.
admin_client = Client(
    username=admin_username,
    ***REMOVED***=admin_***REMOVED***,
    instance_types=[Training_Instance],
    backend_host=backend_host,
    backend_port=backend_port
)

# Spin up an instance. Set who can participate in the instance.
admin_instance = admin_client.create_instance(
    "Training", 
    Training_Instance.type, 
    [participant1_username, participant2_username]
)
print("Instance ID: {}".format(admin_instance.id))

#### Check security guarantees
Validating the so-called fatquote. This step is crucial for all security guarantees.
This step gets and validates the cryptographic proof from the enclave:

* i)   It proves it is a valid SGX enclave (by checking a certificate).
* ii)  It compares the hash of the enclave code provided by the user to
     an expected value (to verify what code is running in the enclave).
* iii) As part of the proof also a public key is transmitted that allows
     establishing a secure connection into the enclave (as the private
     key is only known to the enclave).
     
As we are using a non-production environment, we whitelist the debug and out_of_data flags.

In [ ]:
admin_instance.validate_fatquote(
    expected_measurement=expected_measurement,
    accept_debug=True,
    accept_group_out_of_date=True
)

The quote is part of the fatquote and provides a detailed fingerprint of the program and state of the remote machine. For example:
* using `flags` we can detect if the CPU is running in un-trusted debug mode
* using `*_snv` we can verify if all security patches have been deployed to the infrastructure
* using `mrenclave` we can attest to the exact program being executed on the remote machine

In [ ]:
# Uncomment to inspect 
# print(admin_instance.quote)

#### Configure instance

In [ ]:
# Set the configuration
configuration = Configuration(
    categories_columns=["x_1", "x_2", "x_3", "x_4"],
    value_column="y",
    ***REMOVED***="***REMOVED***" # the ***REMOVED*** to execute
)

# Create and set public-private keypair for secure communication.
admin_secret = Secret()
admin_instance.set_secret(admin_secret)

# Upload
admin_instance.upload_configuration(configuration)

### PARTICIPANT USERS - Submit Data

In [ ]:
# This function submits for a given participant a data file to the instance.
def participant_submit_data(participant_username, participant_***REMOVED***, instance_id, data_file):

    # Create client
    participant_client = Client(
        username=participant_username,
        ***REMOVED***=participant_***REMOVED***,
        instance_types=[Training_Instance],
        backend_host=backend_host,
        backend_port=backend_port
    )

    # Connect to instance (using ID from the admin user)
    participant_instance = participant_client.get_instance(instance_id)

    # Check security guarantees.
    participant_instance.validate_fatquote(
        expected_measurement=expected_measurement,
        accept_debug=True,
        accept_group_out_of_date=True
    )

    # Create and set public-private keypair for secure communication.
    participant_secret = Secret()
    participant_instance.set_secret(participant_secret)

    # Get data format from the enclave
    data_format = participant_instance.get_data_format()
    print("Data format:\n{}".format(data_format))

    # Load data
    with open(data_file, "rb") as f:
        data = f.read()
        
    print("Loaded data:\n{}".format(data.decode("utf-8")))

    # Submit data
    (ingested_rows, failed_rows) = participant_instance.submit_data(data)
    print("Number of successfully ingested rows: {}, number of failed rows: {}".format(ingested_rows, failed_rows))
    
    return participant_instance

#### PARTICIPANT 1 - Submit data

In [ ]:
participant1_instance = participant_submit_data(
    participant1_username, 
    participant1_***REMOVED***, 
    admin_instance.id, 
    "./test-data/participant1_data.csv"
)

#### PARTICIPANT 2 - Submit Data

In [ ]:
participant2_instance = participant_submit_data(
    participant2_username, 
    participant2_***REMOVED***, 
    admin_instance.id, 
    "./test-data/participant2_data.csv"
)

### ADMIN USER - Start Execution

In [ ]:
admin_instance.start_execution(admin_***REMOVED***)

### PARTICIPANT USERS - Get Results

#### PARTICIPANT 1 - Get results

In [ ]:
results1 = participant1_instance.get_results()
print("Result participant1:")
print(json.dumps(json.loads(results1.coeffs.replace(",\n}", "\n}")), indent=2))

#### PARTICIPANT 2 - Get results

In [ ]:
results2 = participant2_instance.get_results()
print("Result participant2:")
print(json.dumps(json.loads(results2.coeffs.replace(",\n}", "\n}")), indent=2))

### ADMIN USER - Clean Up

In [ ]:
admin_instance.shutdown()
admin_instance.delete()
assert admin_instance.id not in admin_client.get_instances()